# Finite volume methods for the Darcy equation

In this tutorial we present how to solve a Darcy equation with [PorePy](https://github.com/pmgbergen/porepy) by using two finite volume methods: two-point flux approximation (TPFA) and multi-point flux approximation (MPFA).

## Exercise 2

We consider the primal formulation of the Darcy problem: the only unknown is the pressure $p$.
We want to solve the problem on different families of grids.

Let $\Omega$ be the domain of interest with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k=I$ the matrix permeability and $f = 1$ the source term, we want to solve the following problem: find $p$ such that
$$
\nabla \cdot (- \nabla p) = f
\quad \text{in } \Omega
$$
with boundary conditions:
$$
\left\{
\begin{array}{ll}
p = 0 & \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega\\
-\nabla p \cdot \nu = 0  & \text{ on } \partial_{top} \Omega \cup \partial_{bottom} \Omega
\end{array}
\right.
$$
Let $\Omega_0 = [0, 1]^2$ and define the shear matrix $S$ as
$$
S =
\begin{bmatrix}
1 & \tan\theta\\
0 & 1
\end{bmatrix}
$$
with $\theta \in [0, \pi/2)$ the shear angle. We define $\Omega = S \circ \Omega_0$.

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules.

In [2]:
import numpy as np
import scipy.sparse as sps
import porepy as pp
import pygeon as pg

We create now the grid and then apply the shear matrix $S$ before calling the `compute_geometry`. Since, on a Cartesian grid and for $\theta = 0$, the pressure computed with TPFA is equal with the pressure computed with MPFA, we consider a Cartesian grid and study the discrepancy between the two solutions by increasing $\theta$.

In [3]:
dim = 2

# TODO creation of a Cartesian grid
N = 25
sd = 

# define the shear matrix
theta = np.pi / 4
S = np.array([[1, np.tan(theta), 0], [0, 1, 0], [0, 0, 1]])

# TODO distort the grid
sd.nodes = 

sd.compute_geometry()

# represent the grid
pp.plot_grid(sd, plot_2d=True)

SyntaxError: invalid syntax (1109760482.py, line 5)

With the following code we set the data, in particular the permeability tensor and the scalar source.

In [ ]:
key = "flow"

# Permeability
perm = pp.SecondOrderTensor(np.ones(sd.num_cells))

# Boundary conditions: identify boundary faces
b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]

# With the following steps we identify the portions of the boundary
# to impose the boundary conditions

top_bottom = np.logical_or(
    sd.face_centers[1, b_faces] == 0, sd.face_centers[1, b_faces] == 1
)

# Set the labels for the boundary conditions
labels = np.array(["dir"] * b_faces.size)
labels[top_bottom] = "neu"

#TODO enforce BCs
bc = 
bc_val = 

# Collect all parameters in a dictionary
data = {
    pp.PARAMETERS: {
        key: {
            "second_order_tensor": perm,
            "bc_values": bc_val,
            "bc": bc,
        }
    },
    pp.DISCRETIZATION_MATRICES: {key: {}},
}

Once the data are assigned to the grid, we need to construct the discrete problem. In particular, the linear system associated with the equation is given as
$$
A p = b
$$<br>
The right-hand side term $b$ includes the part associated with the scalar source term and with the boundary conditions, we construct first the former.

In [5]:
#TODO construct the rhs from the discretization of the scalar source term

scalar_source = 

Second, we construct the matrix $A$ from the TPFA approximation and the part of the right-hand side associated to the boundary conditions. We can compute the pressure $p$ by solving the linear system.

In [6]:
# TODO construct the lhr and rhs from the discretization of the diffusion operator


# solve the problem
cell_p_tpfa = 

As a post-process, we can construct the Darcy flux on each face of the grid by using the flux matrix applied to the computed pressure. 

In [7]:
# extract the discretization matrices build from TPFA
mat_discr = data[pp.DISCRETIZATION_MATRICES][key]

# reconstruct the flux as post-process
q_tpfa = mat_discr["flux"] @ cell_p_tpfa + mat_discr["bound_flux"] @ bc_val

We perform the same steps with the MPFA scheme: to construct the matrix $A$, and associated right-hand side, then compute the pressure $p$ and as a post-process the flux.

In [8]:
# construct the lhr and rhs from the discretization of the diffusion operator


# solve the problem
cell_p_mpfa = 

# now data contains the discretization matrices build from MPFA

q_mpfa = 

We compute now the discrepancy between the pressure computed with the TPFA and MPFA.

In [21]:
# compute the difference between the two solutions


difference 0.04058891434005472 0.7834670888043777


Let us now reconstruct the $P_0$-vector representation of the flux for visualization purposes.

In [9]:
# to export the flux
mvem = pp.MVEM(key)
mvem.discretize(sd, data)

# construct the P0 flux reconstruction
cell_q_tpfa = mvem.project_flux(sd, q_tpfa, data)
cell_q_mpfa = mvem.project_flux(sd, q_mpfa, data)

We export the solutions to be visualized by [ParaView](https://www.paraview.org/).